<a href="https://colab.research.google.com/github/RiaanDeWinnaar/ASCOM.DSLR.2025/blob/main/gemma3270m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Fine-tune the "unsloth/gemma3 270m" model on the "SimpleStories/SimpleStories", "microsoft/rStar-Coder", "zwhe99/DeepMath-103K", and "Muennighoff/mbpp" datasets using the provided Colab notebook template.

## Setup environment

### Subtask:
Install the necessary libraries including Unsloth and dependencies.


**Reasoning**:
Install the required libraries for fine-tuning the model using pip.



In [2]:
!pip install --upgrade pip
!pip install "unsloth[colab-new]@git+https://github.com/unslothai/unsloth.git"
!pip install xformers trl peft accelerate bitsandbytes datasets transformers
!pip install flash_attn --no-build-isolation
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-gp3o3qz8/unsloth_0ed9de0a09a941afbc170384d63c3fe4
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-gp3o3qz8/unsloth_0ed9de0a09a941afbc170384d63c3fe4
  Resolved https://github.com/unslothai/unsloth.git to commit 658a4703c5b7a3b3d3fc5b6c5b5ddc1e607f3b3b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 59.0 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 19.0 MB/s  0:00:00
  Created wheel for unsloth: filename=unsloth-2025.8.10-py3-none-any.whl size=313539 sha256=1ef64b20a8714

## Load model

### Subtask:
Load the specified models (unsloth/gemma3 270m) using Unsloth.


**Reasoning**:
Import the necessary libraries and load the specified model using Unsloth.



In [5]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any length up to 2048
dtype = None # None for auto detection. Supports torch.float16, torch.bfloat16, and torch.float32
load_in_8bit = True # Use 4bit quantization to reduce memory usage

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-3-270m-it",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_8bit = load_in_8bit,
)

/tmp/ipython-input-2203413844.py:1: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


NotImplementedError: Unsloth currently only works on NVIDIA GPUs and Intel GPUs.

## Load Datasets

### Subtask:
Load the specified datasets (SimpleStories/SimpleStories, microsoft/rStar-Coder, zwhe99/DeepMath-103K, Muennighoff/mbpp).

**Reasoning**:
Load the specified datasets using the `datasets` library.

In [9]:
from datasets import load_dataset

# Load the datasets
dataset_simple_stories = load_dataset("SimpleStories/SimpleStories")
dataset_r_star_coder = load_dataset("microsoft/rStar-Coder", name='synthetic_sft') # Specify the configuration name
dataset_deep_math = load_dataset("zwhe99/DeepMath-103K")
dataset_mbpp = load_dataset("Muennighoff/mbpp")

print("SimpleStories/SimpleStories dataset loaded:")
print(dataset_simple_stories)
print("\nmicrosoft/rStar-Coder dataset loaded:")
print(dataset_r_star_coder)
print("\nzwhe99/DeepMath-103K dataset loaded:")
print(dataset_deep_math)
print("\nMuennighoff/mbpp dataset loaded:")
print(dataset_mbpp)

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/103022 [00:00<?, ? examples/s]

The repository for Muennighoff/mbpp contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Muennighoff/mbpp.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating test split: 0 examples [00:00, ? examples/s]

SimpleStories/SimpleStories dataset loaded:
DatasetDict({
    train: Dataset({
        features: ['story', 'topic', 'theme', 'style', 'feature', 'grammar', 'persona', 'initial_word_type', 'initial_letter', 'word_count', 'character_count', 'num_paragraphs', 'avg_word_length', 'avg_sentence_length', 'flesch_reading_ease', 'flesch_kincaid_grade', 'dale_chall_readability_score', 'num_stories_in_completion', 'expected_num_stories_in_completion', 'generation_id', 'model'],
        num_rows: 2115696
    })
    test: Dataset({
        features: ['story', 'topic', 'theme', 'style', 'feature', 'grammar', 'persona', 'initial_word_type', 'initial_letter', 'word_count', 'character_count', 'num_paragraphs', 'avg_word_length', 'avg_sentence_length', 'flesch_reading_ease', 'flesch_kincaid_grade', 'dale_chall_readability_score', 'num_stories_in_completion', 'expected_num_stories_in_completion', 'generation_id', 'model'],
        num_rows: 21371
    })
})

microsoft/rStar-Coder dataset loaded:
DatasetDi

## Prepare Data

### Subtask:
Preprocess and tokenize the loaded datasets for training with the model.

**Reasoning**:
Define a formatting function to prepare the data in a conversational format suitable for the model and apply it to each dataset.

In [12]:
def formatting_function(examples):
    texts = []
    # When batched=True, examples is a dictionary of lists
    # We need to iterate through the examples by index
    for i in range(len(examples[list(examples.keys())[0]])):
        example = {key: examples[key][i] for key in examples.keys()}

        # SimpleStories/SimpleStories has 'story' and other metadata
        if 'story' in example:
            text = f"Below is a story:\n{example['story']}"
        # microsoft/rStar-Coder has 'question', 'response', 'code'
        elif 'question' in example and 'response' in example and 'code' in example:
            text = f"Below is a programming question:\n{example['question']}\nHere is the response:\n{example['response']}\nHere is the code:\n{example['code']}"
        # zwhe99/DeepMath-103K has 'question', 'final_answer', 'r1_solution_1', etc.
        elif 'question' in example and 'final_answer' in example:
            text = f"Below is a math question:\n{example['question']}\nHere is the final answer:\n{example['final_answer']}"
            if 'r1_solution_1' in example and example['r1_solution_1']:
                text += f"\nHere is a solution:\n{example['r1_solution_1']}"
        # Muennighoff/mbpp has 'text', 'code', 'test_list', etc.
        elif 'text' in example and 'code' in example:
            text = f"Below is a programming problem:\n{example['text']}\nHere is the code solution:\n{example['code']}"
        else:
            # Handle other potential structures or skip
            text = "" # Or a default message

        # Add EOS token
        texts.append(text + tokenizer.eos_token)
    return { "text" : texts }

# Apply formatting to each dataset
formatted_dataset_simple_stories = dataset_simple_stories.map(formatting_function, batched = True)
formatted_dataset_r_star_coder = dataset_r_star_coder.map(formatting_function, batched = True)
formatted_dataset_deep_math = dataset_deep_math.map(formatting_function, batched = True)
formatted_dataset_mbpp = dataset_mbpp.map(formatting_function, batched = True)

# Combine datasets (optional, but often done for multi-dataset fine-tuning)
# You might need to select specific splits (e.g., 'train') before concatenating
from datasets import concatenate_datasets

combined_dataset = concatenate_datasets([
    formatted_dataset_simple_stories["train"],
    formatted_dataset_r_star_coder["train"], # Assuming rStar-Coder has a train split
    formatted_dataset_deep_math["train"],
    formatted_dataset_mbpp["test"], # mbpp only has a test split in the loaded version
])

print("\nCombined dataset:")
print(combined_dataset)

Map:   0%|          | 0/2115696 [00:00<?, ? examples/s]

Map:   0%|          | 0/21371 [00:00<?, ? examples/s]

Map:   0%|          | 0/398183 [00:00<?, ? examples/s]

Map:   0%|          | 0/103022 [00:00<?, ? examples/s]

Map:   0%|          | 0/974 [00:00<?, ? examples/s]


Combined dataset:
Dataset({
    features: ['story', 'topic', 'theme', 'style', 'feature', 'grammar', 'persona', 'initial_word_type', 'initial_letter', 'word_count', 'character_count', 'num_paragraphs', 'avg_word_length', 'avg_sentence_length', 'flesch_reading_ease', 'flesch_kincaid_grade', 'dale_chall_readability_score', 'num_stories_in_completion', 'expected_num_stories_in_completion', 'generation_id', 'model', 'text', 'question', 'seed_question', 'seed_source', 'response', 'code', 'final_answer', 'difficulty', 'r1_solution_1', 'r1_solution_2', 'r1_solution_3', 'task_id', 'test_list', 'test_setup_code', 'challenge_test_list'],
    num_rows: 2617875
})


## Configure Training

### Subtask:
Set up LoRA adapters and training arguments for fine-tuning.

**Reasoning**:
Attach LoRA adapters to the model and define the training parameters using `TrainingArguments` from the `transformers` library and `SFTTrainer` from the `trl` library.

In [1]:
from peft import LoraConfig
from transformers import TrainingArguments
from trl import SFTTrainer

# Setup LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number from 8 to 64
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any number from 0 to 0.6
    bias = "none",    # Supports "none", "all", "lora_only"
    use_gradient_checkpointing = "current",
    random_state = 3407,
    use_rslora = False, # We support RS LoRA
    loftq_config = None, # And LoftQ
)

# Define training arguments
training_arguments = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 5,
    max_steps = 60, # Adjust based on your dataset size and desired training length
    learning_rate = 2e-4,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 1,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 3407,
    output_dir = "outputs",
)

# Initialize SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = combined_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_batch_size = 2,
    args = training_arguments,
)

ModuleNotFoundError: No module named 'trl'